In [15]:
import cv2
import os
import time
import csv
import warnings
import pickle

import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


In [5]:
desire_fps = 15
num_frames = 15

original_width, original_height = 500, 500
small_width, small_height = 150, 150

In [16]:
with open('../model/v1_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
camera = cv2.VideoCapture("http://192.168.1.26:5000/video")

frames = []
detected = []

if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()

    while camera.isOpened():
        ret, frame = camera.read()

        if ret:

            frame = cv2.resize(frame, (original_width, original_height))

            if len(frames) < num_frames:
                frames.append(cv2.resize(frame, (small_width, small_height)))

                frame_detected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_detected.flags.writeable = False

                results = holistic.process(frame_detected)

                frame_detected.flags.writeable = True
                frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_RGB2BGR)

                temporary_list = []
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE])
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EYE])
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EYE])
                
                detected.append(temporary_list)
            else:
                frames.pop(0)
                detected.pop(0)

                frames.append(cv2.resize(frame, (small_width, small_height)))

                frame_detected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_detected.flags.writeable = False

                results = holistic.process(frame_detected)
                
                frame_detected.flags.writeable = True
                frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_RGB2BGR)
                
                temporary_list = []
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE])
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EYE])
                temporary_list.append(results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EYE])
                
                detected.append(temporary_list)
            
            if len(frames) >= num_frames:
                stacked1_frames = np.vstack((frames[0], frames[1], frames[2]))
                stacked2_frames = np.vstack((frames[3], frames[4], frames[5]))
                stacked3_frames = np.vstack((frames[6], frames[7], frames[8]))
                stacked4_frames = np.vstack((frames[9], frames[10], frames[11]))
                stacked5_frames = np.vstack((frames[12], frames[13], frames[14]))

                stacked1_frames = cv2.resize(stacked1_frames, (small_width, original_height))
                stacked2_frames = cv2.resize(stacked2_frames, (small_width, original_height))
                stacked3_frames = cv2.resize(stacked3_frames, (small_width, original_height))
                stacked4_frames = cv2.resize(stacked4_frames, (small_width, original_height))
                stacked5_frames = cv2.resize(stacked5_frames, (small_width, original_height))
                
                final_layout = np.hstack((frame, stacked1_frames, stacked2_frames, stacked3_frames, stacked4_frames, stacked5_frames))
            else:
                final_layout = frame

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            flatten_detected = [landmark for sublist in detected for landmark in sublist]

            landmark_list = landmark_pb2.NormalizedLandmarkList()
            landmark_list.landmark.extend(flatten_detected)


            try:
                lv = landmark_list.landmark
                motion_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in lv]).flatten()) 
                
                motion_detected = pd.DataFrame([motion_row])
                motion_class = model.predict(motion_detected)[0]

                warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
                
                cv2.putText(final_layout, f'Class: {motion_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            except Exception as e:
                pass


            cv2.imshow("Camera Feed", final_layout)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
camera.release()
cv2.destroyAllWindows()



Camera is ready. Press 'q' to quit.


I0000 00:00:1748079072.721293   52916 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748079072.747776   63862 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1748079072.996790   63851 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748079073.164845   63852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748079073.176363   63856 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748079073.176751   63853 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

KeyboardInterrupt: 